In [2]:
import os
import PyPDF2

# Directory containing your Bangla text files and PDFs
#Jim Macbook
#directory = "/Users/jim/Documents/Jupyter/CSE498R/498R_20_AUTHORS_WORK"
#Jim PC
directory = r"D:\Study\2025 Summer\CSE 498R\498R_20_AUTHORS_WORK"
# Sample size (number of words per sample)
SAMPLE_SIZE = 500

# Function to extract text from a PDF file
def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text() or ""  # Extract text, handle None case
    return text

# Function to process a single file (txt or pdf) and extract samples
def extract_samples(file_path, file_type):
    if file_type == "txt":
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
    elif file_type == "pdf":
        content = extract_text_from_pdf(file_path)
    
    # Split into words
    words = content.split()  # Split by whitespace (adjust if needed for Bangla)

    # If the file has fewer than 500 words, return it as one sample
    if len(words) <= SAMPLE_SIZE:
        return [content.strip()]

    # Otherwise, split into samples of 500 words
    samples = []
    for i in range(0, len(words), SAMPLE_SIZE):
        sample_words = words[i:i + SAMPLE_SIZE]
        sample_text = " ".join(sample_words)
        samples.append(sample_text)
    return samples

# Process all text and PDF files in the directory
all_samples = []
file_count = 0

for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if filename.endswith(".txt"):
        samples = extract_samples(file_path, "txt")
        all_samples.extend(samples)
        file_count += 1
        print(f"Processed {filename} (txt): {len(samples)} samples extracted")
    elif filename.endswith(".pdf"):
        samples = extract_samples(file_path, "pdf")
        all_samples.extend(samples)
        file_count += 1
        print(f"Processed {filename} (pdf): {len(samples)} samples extracted")

# Summary
total_samples = len(all_samples)
print(f"\nTotal number of files processed: {file_count}")
print(f"Total number of samples extracted: {total_samples}")


Processed অ-আ-ক-খুনের কাঁটা – নারায়ণ সান্যাল.txt (txt): 63 samples extracted
Processed অগ্নিপরীক্ষা – আশাপূর্ণা দেবী.txt (txt): 73 samples extracted
Processed অগ্নিবীণা – কাজী নজরুল ইসলাম.txt (txt): 44 samples extracted
Processed অগ্রন্থিত কবিতা – জীবনানন্দ দাশ.txt (txt): 17 samples extracted
Processed অগ্রন্থিত লেখার সংকলন – তসলিমা নাসরিন.txt (txt): 329 samples extracted
Processed অঘোরে ঘুমিয়ে শিব – দেবারতি মুখোপাধ্যায়.txt (txt): 123 samples extracted
Processed অচল পদাবলী – নির্মলেন্দু গুণ.txt (txt): 1 samples extracted
Processed অতনুর দেশ – কাজী নজরুল ইসলাম.txt (txt): 1 samples extracted
Processed অতলে অন্তরীণ – তসলিমা নাসরিন.txt (txt): 1 samples extracted
Processed অনড় দাঁড়ালাম – তসলিমা নাসরিন.txt (txt): 140 samples extracted
Processed অনন্ত বরফবীথি – নির্মলেন্দু গুণ.txt (txt): 1 samples extracted
Processed অনীশের সেরা ১০১ – অনীশ দেব.txt (txt): 350 samples extracted
Processed অপঘাত – গোলাম মওলা নঈম.txt (txt): 75 samples extracted
Processed অপারেশন তেলআবিব – ১ – সাইমুম সিরিজ #১.

In [10]:
import requests
import json
import os
import time
from tqdm import tqdm
import random

API_KEY = "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"  # Replace with your actual key
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "<YOUR_SITE_URL>",   # Optional
    "X-Title": "<YOUR_SITE_NAME>"        # Optional
}

# Base save directory
base_dir = r"D:\Study\2025 Summer\CSE 498R\generated"

# Create category-specific directories
os.makedirs(os.path.join(base_dir, "poems"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "novels"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "stories"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "dramas"), exist_ok=True)

# Prompts per category
poem_prompts = [
    "একটি বাংলা কবিতা লিখুন।",
    "বাংলা ভাষায় একটি অনুপ্রেরণামূলক কবিতা লিখুন।",
    "একটি আবেগময় বাংলা কবিতা লিখুন।",
    "একটি বাংলা কবিতা লিখুন যা জীবনের বাস্তবতা তুলে ধরে।",
    "বাংলা ভাষায় একটি ছন্দময় কবিতা লিখুন।",
    "একটি বাংলা কবিতা লিখুন যেখানে অনুভূতির প্রকাশ আছে।",
    "বাংলায় একটি চিন্তাশীল কবিতা লিখুন।"
]

novel_prompts = [
    "একটি বাংলা উপন্যাস অনুচ্ছেদ লিখুন।",
    "বাংলা ভাষায় একটি কল্পনাভিত্তিক উপন্যাস লিখুন।",
    "একটি বাংলা উপন্যাসের লিখুন যেখানে মূল চরিত্রের মানসিক পরিবর্তন দেখা যায়।",
    "একটি বাংলা উপন্যাস অনুচ্ছেদ লিখুন যেখানে টান টান উত্তেজনা আছে।",
    "বাংলা ভাষায় একটি চরিত্রনির্ভর উপন্যাস লিখুন।"
]

story_prompts = [
    "একটি বাংলা ছোটগল্প লিখুন।",
    "বাংলা ভাষায় একটি শিক্ষামূলক গল্প লিখুন।",
    "একটি বাংলা কল্পনাভিত্তিক গল্প লিখুন।",
    "বাংলা ভাষায় একটি নৈতিক শিক্ষা ভিত্তিক গল্প লিখুন।",
    "একটি বাংলা গল্প লিখুন যা পাঠকদের ভাবাবে।",
    "বাংলা ভাষায় একটি হৃদয়স্পর্শী ছোটগল্প লিখুন।"
]

drama_prompts = [
    "একটি বাংলা নাটকের দৃশ্য লিখুন।",
    "বাংলা ভাষায় একটি নাটকের সংলাপ লিখুন।",
    "একটি নাটকের অংশ লিখুন যেখানে দুই চরিত্রের মধ্যে দ্বন্দ্ব রয়েছে।",
    "বাংলা ভাষায় একটি মঞ্চ নাটকের সংক্ষিপ্ত অংশ লিখুন।",
    "একটি নাটকের সূচনা অংশ লিখুন যেখানে মূল চরিত্র দর্শকদের পরিচিত হয়।",
    "একটি নাটকের অংশ লিখুন যেখানে চরিত্ররা গুরুত্বপূর্ণ সিদ্ধান্ত নিচ্ছে।"
]

# Text generation function
def generate_text(prompt):
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers=HEADERS,
        data=json.dumps({
            "model": "shisa-ai/shisa-v2-llama3.3-70b:free",
            "messages": [{"role": "user", "content": prompt}]
        })
    )

    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        print("Error:", response.status_code, response.text)
        return None

# Sample generation function per category
def generate_samples(sample_type, prompts, count):
    save_dir = os.path.join(base_dir, f"{sample_type}s")
    os.makedirs(save_dir, exist_ok=True)

    for i in tqdm(range(count), desc=f"Generating {sample_type}s"):
        prompt = random.choice(prompts)
        text = generate_text(prompt)

        if text:
            file_path = os.path.join(save_dir, f"{sample_type}_{i+1}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(text.strip())
        else:
            time.sleep(5)

        time.sleep(1)

# Example usage:
generate_samples("poem", poem_prompts, 10)
generate_samples("novel", novel_prompts, 10)
generate_samples("story", story_prompts, 10)
generate_samples("drama", drama_prompts, 10)

Generating dramas:  40%|█████████████████████████▌                                      | 4/10 [05:37<08:25, 84.32s/it]


KeyError: 'choices'